<a href="https://colab.research.google.com/github/rainmaker29/autoencoder-img-res/blob/master/img_res_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPooling2D,Dropout,Conv2DTranspose
from tensorflow.keras.layers import UpSampling2D,add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# Encoder

In [0]:
input_img = Input(shape=(256,256,3))

l1 = Conv2D(64,(3,3),padding="same",activation='relu',activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(64,(3,3),padding="same",activation='relu',activity_regularizer=regularizers.l1(10e-10))(l1)
l3= MaxPooling2D(padding='same')(l2)
l4 = Dropout(0.3)(l3)
l5 = Conv2D(128, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
l6 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l5)
l7 = MaxPooling2D(padding='same')(l6)
l8 = Conv2D(256, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l7)
encoder= Model(input_img,l8)            

In [5]:
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 128, 128)     1475

# Decoder


In [0]:

input_img = Input(shape=(256, 256, 3))
l1 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)

l3 = MaxPooling2D(padding='same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(128, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)

l6 = MaxPooling2D(padding='same')(l5)
l7 = Conv2D(256, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)

l8 = UpSampling2D()(l7)
l9 = Conv2D(128, (3, 3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l8)
l10 = Conv2D(128, (3, 3), padding='same', activation='relu',
             activity_regularizer=regularizers.l1(10e-10))(l9)

l11 = add([l5, l10])
l12 = UpSampling2D()(l11)
l13 = Conv2D(64, (3, 3), padding='same', activation='relu',
             activity_regularizer=regularizers.l1(10e-10))(l12)
l14 = Conv2D(64, (3, 3), padding='same', activation='relu',
             activity_regularizer=regularizers.l1(10e-10))(l13)

l15 = add([l14, l2])

decoder = Conv2D(3,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l15)



# Autoencoder

In [0]:
autoencoder = Model(input_img,decoder)
#fenn
autoencoder_hfenn = Model(input_img,decoder)

In [11]:
autoencoder.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_15[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_16[0][0]                  
____________________________________________________________________________________________

In [0]:
autoencoder.compile(optimizer='adadelta',loss='mean_squared_error')


# Training

In [0]:
import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import numpy as np

def train_batches(just_load=False):
  batches=5
  batch=0  # Counter - no. of images in current batch,inc and then resets per batch
  batch_nb=0 #batch index

  max_batches=-1

  epochs=4



  images=[]
  x_train_n=[]
  x_train_down=[]

  x_train_n2=[] #resulting high res dataset
  x_train_down2=[] #resulting low res dataset

  for root, dirnames, filenames in os.walk("/drive/My Drive/image_resolution/"):
    for filename in filenames:
      if re.search("\.(jpg|jpeg|JPEG|png|bmp|tiff)$", filename):
        if batch_nb == max_batches:
          # If we limit the number of batches, just return earlier
          return x_train_n2, x_train_down2
        filepath = os.path.join(root, filename)
        image = pyplot.imread(filepath)
        if len(image.shape) > 2:
          image_resized = resize(image, (256, 256)) # Resize the image so that every image is the same size
          x_train_n.append(image_resized) # Add this image to the high res dataset
          x_train_down.append(rescale(rescale(image_resized, 0.5), 2.0)) # Rescale it 0.5x and 2x so that it is a low res image but still has 256x256 resolution
          batch += 1
          if batch == batches:
            batch_nb += 1

            x_train_n2 = np.array(x_train_n)
            x_train_down2 = np.array(x_train_down)
                        
            if just_load_dataset:
              return x_train_n2, x_train_down2
                        
            print('Training batch', batch_nb, '(', batches, ')')

            autoencoder.fit(x_train_down2, x_train_n2,
                            epochs=ep,
                            batch_size=10,
                            shuffle=True,
                            validation_split=0.15)
                    
            x_train_n = []
            x_train_down = []
                    
            batch = 0

  return x_train_n2, x_train_down2


# Loading dataset

In [0]:
x_train_n, x_train_down = train_batches(just_load=True)

In [0]:
autoencoder.load_weights("/content/drive/My Drive/img_res/sr.img_net.mse_hfenn.final_model5_2.no_patch.weights.best.hdf5")

# Predictions

In [0]:
encoder.load_weights('/content/drive/My Drive/img_res/encoder_weights.hdf5')

In [38]:
encoded_imgs = encoder.predict(x_train_down)

IndexError: ignored